In [ ]:
import os
from dotenv import load_dotenv
from IPython.display import display,Markdown,update_display
from openai import OpenAI
import gradio as gr
import json

In [ ]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
model = 'gpt-4o-mini'
openai = OpenAI()

In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

get_ticket_price('london')

Tool get_ticket_price called for london


'$799'

In [ ]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
book_ticket_function = {
    "name": "book_ticket",
    "description": "Book a return flight to the specified destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to book a ticket to"
            },
            "passenger_name": {
                "type": "string",
                "description": "The full name of the passenger"
            },
            "travel_date": {
                "type": "string",
                "description": "Date of travel in YYYY-MM-DD format"
            }
        },
        "required": ["destination_city", "passenger_name", "travel_date"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [
    {'type': 'function', 'function': price_function},
    {'type': 'function', 'function': book_ticket_function}
]


In [ ]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    tool_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    if tool_name == "get_ticket_price":
        city = arguments.get('destination_city')
        price = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city, "price": price}),
            "tool_call_id": tool_call.id
        }
        return response, city

    elif tool_name == "book_ticket":
        city = arguments["destination_city"]
        name = arguments["passenger_name"]
        date = arguments["travel_date"]
        print(f"Tool book_ticket called for {name} to {city} on {date}")
        confirmation = f"Booking confirmed for {name} to {city.title()} on {date}."
        response = {
            "role": "tool",
            "content": json.dumps({
                "destination_city": city,
                "passenger_name": name,
                "travel_date": date,
                "confirmation": confirmation
            }),
            "tool_call_id": tool_call.id
        }
        return response, None

    else:
        raise ValueError(f"Unknown tool: {tool_name}")


In [ ]:
def chat(message,history):
    messages = [{'role':'system','content':system_message}] + history + [{'role':'user','content':message}]
    response = openai.chat.completions.create(model=model,messages=messages,tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response,city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=model,messages=messages)

    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


Tool book_ticket called for Marwan Ashraf to London on 2025-09-20
Tool get_ticket_price called for london
